In [1]:
using CUDA
using CUDA.CUSPARSE
using LinearAlgebra
using Laplacians
using BenchmarkTools
using SparseArrays
using Profile, ProfileSVG

# Graph that is used to compute
For now, using the `pure_random_graph()` API in `Laplacians` library to generate an input graph to keep things simple. My goal at this stage is familiar with Julia and write a working GPU accelerated PCG implementation.

In [2]:
# generate a random graph for now: to keep the framework simple
G = pure_random_graph(500000)

500000×500000 SparseMatrixCSC{Float64, Int64} with 999998 stored entries:
⢻⡲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⢳⡀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⢳⡀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠲⢤⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀

# compute UR
`UR` depends on a random projection matrix. To eliminate the randomness, UR is precomputed.

In [3]:
# need to fix the random projection matrix
JLfac = 4.0

n = size(G,1)
k = round(Int, JLfac*log(n)) # number of dims for JL, natrual log
U = wtedEdgeVertexMat(G) # equal to W, mxn
m = size(U,1)

R = randn(Float64, m,k) # equal to Q, JL projection matrix, mxk

UR = U'*R; # nxk

500000×52 Matrix{Float64}:
  1.56352    0.0264318  -0.405654   …  -2.15465     -0.339218    2.29699
 -0.771098   0.291839    1.74057       -0.376628    -1.0792     -3.38268
 -2.44124    1.31332    -0.309925       1.21666     -0.554592   -4.05069
 -1.44046    1.88636    -2.02203        0.244647     1.29985     2.17637
  1.47198   -4.30914    -2.70354        5.36348     -0.654591    0.0805611
 -1.26504    1.63984     1.73258    …   0.486712     0.60848     0.845595
  0.67612   -0.572674    2.35618        0.00995473   1.59809     1.24164
 -1.22466   -2.79176     0.80713       -2.10519      1.50387    -0.241062
  1.81701    0.397472    1.47036        0.510398    -2.92432    -1.4947
 -1.36539   -0.834711    0.158229      -4.98645      0.135314   -0.357961
 -2.73702    1.20562     1.31942    …  -1.9408      -0.427102    0.368647
  1.10546    0.395197   -0.162306       0.378567    -0.0275037   0.080736
  2.26089   -1.20489    -1.07327       -0.0647563   -2.36894     0.0616504
  ⋮             

# The CPU baseline & Ground truth
the following section using the code from yuhan's repo as a CPU baseline as well as the ground truth to verify correctness. tollerance is set to 0.0001 to increase the iteration in each pcg.

In [4]:
# the reference function from Yuhan's repo that uses Laplacian's library
function compute_V(a, UR)
  f = approxchol_lap(a,tol=1e-2); # a is the weight matrix loaded from file, nxn

  V = zeros(n,k)
  print("Time to solve Lap: ")
  @time for i in 1:k
    V[:,i] = f(UR[:,i]) # f is the linear solver, solve for x in Ax=b
  end
  
  return V # V here is the Z in paper
end

compute_V (generic function with 1 method)

In [12]:
Ztruth = compute_V(G, UR)

Time to solve Lap:   1.153664 seconds (1.93 k allocations: 1.550 GiB, 6.10% gc time)


500000×52 Matrix{Float64}:
 -0.136154   0.221333   -0.101415  …  -0.864206   -0.543875  -0.637303
 -1.33927    0.101885    0.94094       0.274264   -0.600522  -2.22005
 -0.496564   0.31435    -0.738116      0.151975   -0.148009  -1.35155
 -2.01569    0.574576    1.47215       0.420305    0.121864  -0.233282
 -1.09486   -0.782095   -0.288486      1.64332    -0.30036   -2.40688
 -0.653569  -0.0810101  -1.21161   …   0.017349    0.272805  -0.0757558
  1.74127   -0.510598   -0.591395      0.0861215   0.381633   0.709085
 -2.00904    0.142638    2.17329      -0.726002    0.188597  -1.00654
 -1.25829   -0.407155    3.32424       1.46801    -0.522331   0.350373
 -0.395821   0.852829    0.687504     -1.16199     0.182291  -1.70919
 -3.02147    1.00814     0.209637  …   0.454204    0.171743  -3.37195
 -0.501783  -1.26335    -2.27705       1.20961     0.949149   0.77482
  0.302677  -0.933872   -2.35225      -1.79625    -0.591203  -0.496129
  ⋮                                ⋱               ⋮    

# GPU accelerated version
the GPU compute_V uses `approxchol_lap_gpu()` (Actually directly using greedy version). The gpu version accelerates the computation inside a loop using gpu, except the preconditioning part.

In [6]:
# some helper function
cudot(cuva, cuvb) = reduce(+, cuva .* cuvb)
cunorm(cuv) = sqrt(reduce(+, cuv .* cuv))


# GPU accelerated PCG algorithm
# mat is assumed to be a CuSparseMatrixCSC
function pcg_gpu(mat::CuSparseMatrixCSC, b::Vector{Tval}, pre::Function;
  tol::Real=1e-6, maxits=Inf, maxtime=Inf, verbose=false, pcgIts=Int[],
  stag_test::Integer=0) where Tval

local al::Tval

n = size(mat,2)

cub = CuVector(b)
nb = cunorm(cub)

# If input vector is zero, quit
if nb == 0
  return zeros(size(b))
end

x = CuVector(zeros(Tval,n)) # move these to GPU
bestx = CuVector(zeros(Tval,n))
bestnr = 1.0

r = copy(cub) # the copy is still an Object on GPU
z = CuVector(pre(Vector(r))) # first move r to CPU, apply precondition, then create an object on GPU
p = copy(z) # p is an object on GPU

rho = cudot(r, z) # make a dot project on GPU, rho is a scala
best_rho = rho
stag_count = 0

t1 = time()

itcnt = 0
while itcnt < maxits
  itcnt = itcnt+1

  q = mat*p # it is a CuSparse * CuVector, result is a CuVector

  pq = cudot(p,q) # both p and q are CuVector, pq is a scala

  if (pq < eps(Tval) || isinf(pq))
    if verbose
      println("PCG Stopped due to small or large pq")
    end
    break
  end

  al = rho/pq # rho and pq are scalas, al is scala

  # the following line could cause slowdown
  if al*norm(p) < eps(Tval)*norm(x)
    if verbose
      println("PCG: Stopped due to stagnation.")
    end
    break
  end

  x = al * p + x; # axpy(al, p, x)
  r = -al * q + r; # axpy(-al, q, r)

  nr = cunorm(r)/nb # nr is scala
  if nr < bestnr
    bestnr = nr
    bestx = x
  end
  if nr < tol #Converged?
      break
  end

  # here is the top of the code in numerical templates

  z = CuVector(pre(Vector(r))) # first move r to CPU, apply precondition, then create an object on GPU

  oldrho = rho
  rho = cudot(z, r) # this is gamma in hypre.

  if rho < best_rho*(1-1/stag_test)
    best_rho = rho
    stag_count = 0
  else
    if stag_test > 0
      if best_rho > (1-1/stag_test)*rho
        stag_count += 1
        if stag_count > stag_test
          println("PCG Stopped by stagnation test ", stag_test)
          break
        end
      end
    end
  end

  if (rho < eps(Tval) || isinf(rho))
    if verbose
      println("PCG Stopped due to small or large rho")
    end
    break
  end

  # the following would have higher accuracy
  #       rho = sum(r.^2)

  beta = rho/oldrho
  if (beta < eps(Tval) || isinf(beta))
    if verbose
      println("PCG Stopped due to small or large beta")
    end
    break
  end

  p = z + beta * p # p = z + beta*p

  if (time() - t1) > maxtime
      if verbose
          println("PCG New stopped at maxtime.")
      end
      break
  end

end

if verbose
  println("PCG stopped after: ", round((time() - t1),digits=3), " seconds and ", itcnt, " iterations with relative error ", (norm(r)/norm(b)), ".")
end

if length(pcgIts) > 0
  pcgIts[1] = itcnt
end


return Vector(bestx) # transfer back from GPU
end

pcg_gpu (generic function with 1 method)

In [7]:
# this version using GPU pcg solver.
function approxchol_lap_gpu(a::SparseMatrixCSC;
  tol::Real=1e-6,
  maxits=1000,
  maxtime=Inf,
  verbose=false,
  pcgIts=Int[],
  params=ApproxCholParams())

  tol_ =tol
  maxits_ =maxits
  maxtime_ =maxtime
  verbose_ =verbose
  pcgIts_ =pcgIts

  t1 = time()

  la = lap(a) # a hit !?

  llmat = Laplacians.LLmatp(a)
  ldli = Laplacians.approxChol(llmat)
  F(b) = Laplacians.LDLsolver(ldli, b)

  if verbose
    println("Using greedy degree ordering. Factorization time: ", time()-t1)
    println("Ratio of operator edges to original edges: ", 2 * length(ldli.fval) / nnz(a))
  end

  if verbose
      println("ratio of max to min diagonal of laplacian : ", maximum(diag(la))/minimum(diag(la)))
  end

  # Create Objects on GPU
  cula = CuSparseMatrixCSC(la)

  # can only pass la to GPU before computer happens
  f(b;tol=tol_,maxits=maxits_, maxtime=maxtime_, verbose=verbose_, pcgIts=pcgIts_) = pcg_gpu(cula, b .- mean(b), F, tol=tol, maxits=maxits, maxtime=maxtime, pcgIts=pcgIts, verbose=verbose, stag_test = params.stag_test)

end

approxchol_lap_gpu (generic function with 1 method)

In [8]:
function compute_V_gpu(a, JLfac=4.0)
  f = approxchol_lap_gpu(a,tol=1e-2); # a is the weight matrix loaded from file, nxn

  V = zeros(n,k)
  print("Time to solve Lap: ")
  @time for i in 1:k
    V[:,i] = f(UR[:,i]) # f is the linear solver, solve for x in Ax=b
  end
  
  return V # V here is the Z in paper
end

compute_V_gpu (generic function with 2 methods)

In [13]:
Zgpu = compute_V_gpu(G, UR)

Time to solve Lap:   0.589542 seconds (37.73 k allocations: 1.358 GiB, 4.23% gc time)


500000×52 Matrix{Float64}:
 -0.136154   0.221333   -0.101415  …  -0.864206   -0.543875  -0.637303
 -1.33927    0.101885    0.94094       0.274264   -0.600522  -2.22005
 -0.496564   0.31435    -0.738116      0.151975   -0.148009  -1.35155
 -2.01569    0.574576    1.47215       0.420305    0.121864  -0.233282
 -1.09486   -0.782095   -0.288486      1.64332    -0.30036   -2.40688
 -0.653569  -0.0810101  -1.21161   …   0.017349    0.272805  -0.0757558
  1.74127   -0.510598   -0.591395      0.0861215   0.381633   0.709085
 -2.00904    0.142638    2.17329      -0.726002    0.188597  -1.00654
 -1.25829   -0.407155    3.32424       1.46801    -0.522331   0.350373
 -0.395821   0.852829    0.687504     -1.16199     0.182291  -1.70919
 -3.02147    1.00814     0.209637  …   0.454204    0.171743  -3.37195
 -0.501783  -1.26335    -2.27705       1.20961     0.949149   0.77482
  0.302677  -0.933872   -2.35225      -1.79625    -0.591203  -0.496129
  ⋮                                ⋱               ⋮    

# Check result

In [11]:
# as the pcg will exit based on convergence condition, consider the result is correct when result is close
diff = Zgpu - Ztruth
mean(abs.(diff))

3.211340116499415e-16

In [159]:
abs(minimum(diff))

3.717221104926338e-6

In [123]:
cm = CuSparseMatrixCSC(sprandn(100, 100, 0.2))

100×100 CuSparseMatrixCSC{Float64, Int32} with 1982 stored entries:
⠆⠷⠀⣄⠂⡢⠦⠔⡤⡢⠢⢥⠀⠇⡠⠁⢁⠃⣠⡏⠟⠂⢴⠠⢘⡠⢞⠂⢻⠌⠚⡌⠐⠈⢸⢄⢄⡌⠼⠬
⢒⠀⠐⣆⡔⢂⠔⡒⢤⠎⠠⠄⠨⠧⠠⠆⡄⠜⡎⠦⠠⠫⠄⠠⠄⢰⠔⡁⠰⠜⠦⠠⠄⠨⡨⠈⢘⠤⢸⠄
⠢⠣⠀⡐⠴⠇⡡⡠⡤⠦⠠⡈⠮⢆⢐⡋⠄⠥⠄⠇⠠⢒⢢⢤⠠⢀⠴⢀⠤⢀⠐⠂⡬⠊⠉⠄⠨⠤⠁⠘
⠂⠓⠡⡀⢔⠦⠀⠅⠄⡜⢈⡀⠀⠀⠉⡄⠅⡄⢨⠄⠐⢄⡀⢲⠔⢰⠀⢀⠠⢀⢠⠂⠭⠂⢭⢀⡃⡍⠲⠅
⠜⠀⢄⠄⠂⠴⠔⠥⢔⢤⠄⠅⡌⠉⠀⢭⠁⠡⢀⢔⠈⠰⢌⠐⣐⠁⠠⡀⠠⠂⠨⠭⡬⠤⠒⠌⠐⠅⡢⠁
⠬⠂⠰⡅⡁⠙⠤⡀⠀⠮⢼⠤⡢⢑⢲⢄⣀⡷⡙⠈⢔⠪⠤⠁⡝⠀⢦⠀⡶⡀⠐⠄⡤⠇⢨⡗⠂⠋⢈⡂
⠀⠎⠘⠤⠀⡦⠾⡁⠀⠎⡈⡀⡐⠌⠁⡀⠐⢆⢘⡐⢅⠀⠜⢴⢂⢑⢅⡸⢡⣊⢻⠆⠠⠤⣡⢸⢨⡈⠲⠬
⠦⠠⢄⠀⢘⠗⠀⠠⠆⣆⠁⠤⠤⠄⠔⣚⡁⡢⡀⠗⡦⠡⠡⠤⡉⠅⢨⠢⢄⠀⡠⡌⣔⠂⣠⣃⣗⠐⢤⠃
⠁⠀⢡⡇⠀⠀⠐⠳⠣⡦⠄⢍⢓⠦⠲⠲⠤⠣⠀⡍⡨⡠⠠⠤⣥⠀⢠⠈⠼⠅⣠⠪⢄⠠⠸⠄⠋⠡⠀⠆
⠀⠑⣔⠅⠢⠢⢀⠐⠡⢀⠆⠄⠠⡲⠄⡃⢨⡼⠁⠅⣸⠠⠏⠤⢡⠐⠥⠄⢀⠀⢪⠅⡼⠤⢸⠵⢮⠨⢬⠰
⡖⡧⢀⢎⢠⣃⢂⠗⠑⠋⢢⠷⠀⠡⡂⠷⡀⠂⠤⢕⠋⡃⠈⠀⠴⠋⠣⡰⢚⠀⠎⢦⠒⡚⠐⠧⠔⠐⠊⢐
⠠⠁⢀⢖⠘⠂⠢⠒⠂⢶⢆⡅⠀⠠⣔⠅⠂⡅⢈⢄⡐⠒⢤⠒⠊⠘⡑⡐⢐⢐⠁⠳⣘⣐⢰⠔⡰⢒⠚⠊
⢀⠒⠀⡎⢀⠢⠠⡀⠂⢒⡔⠅⠄⣲⠁⡄⠀⢪⠈⠜⡒⠀⢈⠐⣔⠆⠢⠵⢸⠪⢰⠀⠫⢑⣈⠀⠚⢺⢖⢀
⠘⠎⠀⡺⠪⠑⠐⠂⠄⠓⠂⠒⠊⣦⠐⠆⠐⠊⡀⡃⠰⠀⡨⠃⢘⡤⢚⡄⣣⡐⠞⢁⠒⠕⠘⠂⢌⢨⢐⠆
⠐⡲⠔⡊⠖⠋⢊⠆⠑⠐⡀⠡⠆⠁⡀⠀⠐⠄⠂⠃⠔⠀⠌⡁⠀⠪⠐⠐⠒⠃⠊⠲⡷⢐⠐⣂⠲⡣⠲⠰
⡄⣐⢰⠄⠀⡈⢘⠔⢁⡔⡚⡤⠒⠒⢂⢃⠁⠂⠘⣖⣲⠊⢙⠀⡊⠁⢢⠩⠈⠂⢠⠀⠲⠀⠁⡦⠒⠂⠊⠒
⠀⡇⠂⠒⠀⠠⡂⡂⠜⡃⡀⠂⡠⠁⠀⢠⡛⠠⠂⢓⢢⠠⠪⢑⠰⡆⠠⠂⠨⠩⠒⠀⠰⠂⠲⢂⠜⠀⠺⣦
⡒⡃⠓⠆⢊⠛⠀⠐⢀⠅⠁⠀⠃⠂⠀⣂⠅⢂⡓⣊⢜⠒⠰⠂⢇⡈⠐⢂⠫⠐⠐⠒⠊⢂⣸⡀⡛⠊⢠⠠
⡂⠲⠨⡂⡀⠡⠐⠒⡓⡼⠠⠂⠍⢂⠐⠂⠲⡄⢄⢀⢀⠁⡐⠰⠲⡹⠘⠀⠸⠙⢆⡺⢰⠄⠚⡠⡈⠠⢈⠒
⠁⡀⡆⡆⣷⢧⢚⡔⡤⠵⠄⡅⠘⡠⢀⠊⠤⡶⢔⠃⢈⠂⢀⠀⠃⠤⢻⠌⢂⠋⢲⠴⠞⠁⠀⡊⡚⠀⢈⠀

In [12]:
for i in 1:10000; cm * cv; end

In [106]:
reduce(+, cv .* cv) # dot product

26.215722326703602

In [101]:
tt = Vector(cv)

100-element Vector{Float64}:
 0.7124631363372693
 0.265700079127287
 0.26485347658042224
 0.6678531149154389
 0.9922126352602408
 0.3667024377203091
 0.4693941714765961
 0.2703520496144789
 0.2260796532265591
 0.04660779725173281
 ⋮
 0.6692765766444105
 0.09336537126440936
 0.5908781827065405
 0.9657135359954367
 0.5803623647514778
 0.8744399568648321
 0.1712150273251376
 0.50326781043447
 0.10453535698078131

In [142]:
cudot

cudot (generic function with 1 method)

In [122]:
copy(cv)

100-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 0.7124631363372693
 0.265700079127287
 0.26485347658042224
 0.6678531149154389
 0.9922126352602408
 0.3667024377203091
 0.4693941714765961
 0.2703520496144789
 0.2260796532265591
 0.04660779725173281
 ⋮
 0.6692765766444105
 0.09336537126440936
 0.5908781827065405
 0.9657135359954367
 0.5803623647514778
 0.8744399568648321
 0.1712150273251376
 0.50326781043447
 0.10453535698078131

In [110]:
sqrt(dot(tt, tt))

5.120129131838728

In [114]:
typeof(UR)

Matrix{Float64} (alias for Array{Float64, 2})

In [115]:
typeof(G)

SparseMatrixCSC{Float64, Int64}

In [128]:
cudot(cuv) = reduce(+, cuv .* cuv)

cudot (generic function with 1 method)

In [133]:
cv * 2*2 + cv

100-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 3.5623156816863464
 1.328500395636435
 1.3242673829021112
 3.3392655745771944
 4.961063176301204
 1.8335121886015453
 2.3469708573829804
 1.3517602480723945
 1.1303982661327954
 0.23303898625866404
 ⋮
 3.3463828832220526
 0.4668268563220468
 2.9543909135327024
 4.8285676799771835
 2.9018118237573893
 4.3721997843241605
 0.856075136625688
 2.51633905217235
 0.5226767849039066